










# Evaluate using Azure AI Evaluation custom evaluators


## Objective
In this notebook we will demonstrate how to use the target functions with the custom evaluators to evaluate an endpoint.

This tutorial provides a step-by-step guide on how to evaluate a function

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 20 minutes running this sample. 

## About this example

This example demonstrates evaluating a target function using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [ ]:
%pip install azure-ai-evaluation
%pip install azure-identity

### Parameters and imports

In [ ]:
import pandas as pd
import os

from pprint import pprint
from azure.ai.evaluation import evaluate
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

azure_openai_api_version = "<api version>"
azure_openai_deployment = "<your-deployment>"
azure_openai_endpoint = "<your-endpoint>"

## Target function
We will use a simple `endpoint_callback` to get answers to questions from our model. We will use `evaluate` API to evaluate `endpoint_callback` answers

`endpoint_callback` needs following environment variables to be set

- AZURE_OPENAI_API_VERSION
- AZURE_OPENAI_DEPLOYMENT
- AZURE_OPENAI_ENDPOINT

In [ ]:
# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_VERSION"] = azure_openai_api_version
os.environ["AZURE_OPENAI_DEPLOYMENT"] = azure_openai_deployment
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_openai_endpoint

In [ ]:
async def endpoint_callback(query: str) -> dict:
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")

    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

    oai_client = AzureOpenAI(
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )

    response_from_oai_chat_completions = oai_client.chat.completions.create(
        messages=[{"content": query, "role": "user"}], model=deployment, max_tokens=500
    )

    response_result = response_from_oai_chat_completions.to_dict()
    return {"query": query, "response": response_result["choices"][0]["message"]["content"]}

## Data
Reading existing dataset which has bunch of questions we can Ask Wiki

In [ ]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

## Running Blocklist Evaluator to understand its input and output

In [ ]:
from blocklist import BlocklistEvaluator

blocklist_evaluator = BlocklistEvaluator(blocklist=["bad", "worst", "terrible"])

blocklist_evaluator(response="New Delhi is Capital of India")

## Run the evaluation

In [ ]:
results = evaluate(
    data="data.jsonl",
    target=blocklist_evaluator,
    evaluators={
        "blocklist": blocklist_evaluator,
    },
)

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])